# Ermittle Wegstrecken f체r Wochentage 

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [4]:
df = pd.read_csv(r"C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\NHTS_trips_processed.csv")

In [5]:
df.head()

ID  Type_day  TRPTRANS  Trip_no  Whyfrom  Whyto  Distance  \
0  300000071         1         3        1        1      4     8.439   
1  300000071         1         3        2        4      1     8.286   
2  300000072         1         6        1        2      1   135.191   
3  300000072         1         6        2        1      2   131.367   
4  300000073         1         3        1        1      2     3.621   

   Trip_duration  Departure_hhmm  Arrival_hhmm  Departure  Arrival  \
0             15            1000          1015        600      615   
1             20            1510          1530        910      930   
2            120             700           900        420      540   
3            150            1800          2030       1080     1230   
4             15             845           900        525      540   

   Departure_t  
0           40  
1           61  
2           28  
3           72  
4           35

## Filtern nach Werktagen und Wegstrecke > 0

In [6]:
filt = (df["Type_day"] < 6) & (df["Distance"] > 0.0)
data = df[filt]

In [7]:
data.head()

ID  Type_day  TRPTRANS  Trip_no  Whyfrom  Whyto  Distance  \
0  300000071         1         3        1        1      4     8.439   
1  300000071         1         3        2        4      1     8.286   
2  300000072         1         6        1        2      1   135.191   
3  300000072         1         6        2        1      2   131.367   
4  300000073         1         3        1        1      2     3.621   

   Trip_duration  Departure_hhmm  Arrival_hhmm  Departure  Arrival  \
0             15            1000          1015        600      615   
1             20            1510          1530        910      930   
2            120             700           900        420      540   
3            150            1800          2030       1080     1230   
4             15             845           900        525      540   

   Departure_t  
0           40  
1           61  
2           28  
3           72  
4           35

## Wegstrecken in 2-dimensionaler Liste Speichern 
wege_ij[i-1][j-1] entspricht der Liste aller Wegstrecken vom Zustand i zum Zustand j

In [8]:
# initialisieren 5x5 Liste 
wege_ij = [[0 for i in range(5)] for j in range(5)]

In [9]:
for i in range(1, 6):
    for j in range(1, 6):
        if j == i:
            continue
        # filtern des Dataframes nach Ausgangs- und Zielzustandskombinationen
        filt = (data["Whyfrom"] == i) & (data["Whyto"] == j)
        # speichern der Liste der Distanzen zwischen den Zust채nden in entsprechendem Feld
        wege_ij[i-1][j-1] = list(data[filt]["Distance"])

### Test

In [10]:
filt = (data["Whyfrom"] == 1) & (data["Whyto"] == 2)
weg_zuhause_arbeit = list(data[filt]["Distance"])

filt = (data["Whyfrom"] == 5) & (data["Whyto"] == 3)
weg_sonstiges_einkaufen = list(data[filt]["Distance"])

In [11]:
(wege_ij[0][1] == weg_zuhause_arbeit) and (wege_ij[4][2] == weg_sonstiges_einkaufen)

True

## Visualisieren der Histogramme

In [12]:
sns.set()
fig, axs = plt.subplots(5, 5, figsize=(20,10))
for i in range(5):
    for j in range(5):
        if i == j:
            continue
        axs[i][j].hist(wege_ij[i][j], bins=range(int(min(wege_ij[i][j])), int(max(wege_ij[i][j])) + 1, 2), density=True)
        axs[i][j].set_xlim(-10, 100)

In [13]:
# initialisieren 5x5 Liste f체r Parameter der Verteilungsfunktion 
shape_ij = [[0 for i in range(5)] for j in range(5)]
scale_ij = [[0 for i in range(5)] for j in range(5)]
loc_ij = [[0 for i in range(5)] for j in range(5)]
pdf_ij = [[0 for i in range(5)] for j in range(5)]
x = np.linspace(0, 500, 1000)

for i in range(5):
    for j in range(5):
        if j == i:
            continue
        shape_ij[i][j], loc_ij[i][j], scale_ij[i][j] = stats.lognorm.fit(wege_ij[i][j], loc=0, scale=5)
        pdf_ij[i][j] = stats.lognorm.pdf(x, s=shape_ij[i][j], loc=loc_ij[i][j], scale=scale_ij[i][j])

In [14]:
fig, axs = plt.subplots(5, 5, figsize=(30,15))
for i in range(5):
    for j in range(5):
        if i==j:
            continue
        axs[i][j].hist(wege_ij[i][j], bins=range(int(min(wege_ij[i][j])), int(max(wege_ij[i][j])) + 1, 2), density=True)
        axs[i][j].set_xlim(-10, 100)
        axs[i][j].plot(x, pdf_ij[i][j])
fig.suptitle("Histogramm und angepasste Lognormalverteilung der Wegstrecken zwischen den Zust채nden");

## Testen samplen der Lognorm Funktion

In [15]:
# sample Werte generieren
sample_test = stats.lognorm.rvs(size=100000, s=shape_ij[0][1], loc=loc_ij[0][1], scale=scale_ij[0][1])

In [16]:
plt.figure()
plt.hist(sample_test, bins=range(int(min(wege_ij[0][1])), int(max(wege_ij[0][1])) + 1, 2), density=True)
plt.gca().set_xlim(-10,100)


(-10, 100)

## Speichern der Funktionsparameter

In [17]:
from pathlib import Path
import pickle
import os

root = os.getcwd()
if not os.path.exists(os.getcwd()+ "\\Simulationsdaten"):
    os.makedirs(root+"\\Simulationsdaten")

save_shape = root+"\\Simulationsdaten"+"\\Verteilung_Wegstrecken_Werktage_Shape.pickle"
save_scale = root+"\\Simulationsdaten"+"\\Verteilung_Wegstrecken_Werktage_Scale.pickle"
save_loc = root+"\\Simulationsdaten"+"\\Verteilung_Wegstrecken_Werktage_Loc.pickle"
save_wege = root+"\\Simulationsdaten"+"\\Liste_Wegstrecken_Werktage.pickle"

pickle.dump(shape_ij, open(save_shape, "wb"))
pickle.dump(scale_ij, open(save_scale, "wb"))
pickle.dump(loc_ij, open(save_loc, "wb"))
pickle.dump(wege_ij, open(save_wege, "wb"))